In [1]:
import pandas as pd 
import ast
import re
import numpy as np
import nltk
import json
from datetime import datetime
from datetime import timedelta
from textblob import TextBlob
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.util import ngrams
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
import pyLDAvis.gensim_models
# import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bandy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bandy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bandy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordne

True

In [3]:
# df = pd.read_csv("/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/train/kickstarter_train.csv")
# df = pd.read_csv("kickstarter_train.csv")
df = pd.read_csv("/Users/bandy/Downloads/data/kickstarter_train.csv")


In [ ]:
# extract main category and subcategory from category variable
# df['subcategory'] = df['category'].apply(lambda x: json.loads(x)['slug'].split("/")[1] if len(json.loads(x)['slug'].split("/")) > 1 else None)
#df['category'] = df['category'].apply(lambda x: json.loads(x)['slug'].split("/")[0])
# df['location'] = df['location'].apply(lambda x: json.loads(x)['country'])

## NLP features

In [5]:
# Text cleaning for: rewards, description, description story, description risks

def clean_text(df):
    def process_rewards(corpus):
        corpus_processed = []
        for row in corpus:
            row_processed = ""
            row = row.replace("\\\n", " ")
            row = ast.literal_eval(row)

            for dict in row:
                row_processed += dict['rewards'].lower() + ' '

            
            row_processed = row_processed.replace("//",'')
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed)

        return corpus_processed
    
    def process_description_story(corpus):
        corpus_processed = []
        for row in corpus:
            row = str(row)
            row_processed = row.replace("\r", " " )
            row_processed = row_processed.replace("\n", " " )
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed if not pd.isnull(row_processed) else "") # handle NA

        return corpus_processed

    df["rewards_processed"] = process_rewards(df["rewards"])
    df["description_processed"] = process_description_story(df["description"])
    df["description_story_processed"] = process_description_story(df["description_story"])
    df["description_risks_processed"] = process_description_story(df["description_risks"])

    return df


In [6]:
class LemmatizeTokenizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, text):
        return [self.lemmatizer.lemmatize(word) for word in word_tokenize(text)]

In [7]:
# need to return the vectorizer to use on the test data to transform it 
def generate_nlp_features(df):
    
    tfidf_feature_names = {}
    # Rewards

    vect_rewards = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )
    

    rewards_processed = pd.Series(df["rewards_processed"])
    tfidf_fit_rewards = vect_rewards.fit(rewards_processed)
    rewards_tfidf_array = tfidf_fit_rewards.transform(rewards_processed).toarray()
    rewards_tfidf_df = pd.DataFrame(rewards_tfidf_array)
    rewards_tfidf_df.columns = list(map(lambda x : "rewards_" + str(x), rewards_tfidf_df.columns))
    df = pd.merge(df, rewards_tfidf_df , left_index=True, right_index=True)
    tfidf_feature_names["rewards"] = list(zip(range(100),vect_rewards.get_feature_names()))
    

    # Description

    vect_description = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    description = pd.Series(df["description_processed"])
    tfidf_fit_description = vect_description.fit(description)
    description_tfidf_array = tfidf_fit_description.transform(description).toarray()
    description_tfidf_df = pd.DataFrame(description_tfidf_array)
    description_tfidf_df.columns = list(map(lambda x : "description_" + str(x), description_tfidf_df.columns))
    df = pd.merge(df, description_tfidf_df , left_index=True, right_index=True)
    tfidf_feature_names["description"] = list(zip(range(100),vect_description.get_feature_names()))

    # Description Story

    vect_description_story = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )
    
    description_story_processed = pd.Series(df["description_story_processed"])
    tfidf_fit_description_story_processed = vect_description_story.fit(description_story_processed)
    description_story_processed_tfidf_array = tfidf_fit_description_story_processed.transform(description_story_processed).toarray()
    description_story_tfidf_df = pd.DataFrame(description_story_processed_tfidf_array)
    description_story_tfidf_df.columns = list(map(lambda x : "description_story_" + str(x), description_story_tfidf_df.columns))
    df = pd.merge(df, description_story_tfidf_df , left_index=True, right_index=True)
    tfidf_feature_names["description_story"] = list(zip(range(100),vect_description_story.get_feature_names()))



    # Description Risks

    vect_description_risks = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    description_risks_processed = pd.Series(df["description_risks_processed"])
    tfidf_fit_description_risks_processed = vect_description_risks.fit(description_risks_processed)
    description_risks_processed_tfidf_array = tfidf_fit_description_risks_processed.transform(description_risks_processed).toarray()
    description_risks_tfidf_df = pd.DataFrame(description_risks_processed_tfidf_array)
    description_risks_tfidf_df.columns = list(map(lambda x : "description_risks_" + str(x), description_risks_tfidf_df.columns))
    df = pd.merge(df, description_risks_tfidf_df , left_index=True, right_index=True)
    tfidf_feature_names["description_risks"] = list(zip(range(100),vect_description_risks.get_feature_names()))

    return (df, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed, tfidf_feature_names)


def nlp_transform_test(df, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed):
    rewards_tfidf_array = tfidf_fit_rewards.transform(df['rewards_processed']).toarray()
    rewards_tfidf_df = pd.DataFrame(rewards_tfidf_array)
    rewards_tfidf_df.columns = list(map(lambda x : "rewards_" + str(x), rewards_tfidf_df.columns))
    df = pd.merge(df, rewards_tfidf_df , left_index=True, right_index=True)
    
    description_tfidf_array = tfidf_fit_description.transform(df['description_processed']).toarray()
    description_tfidf_df = pd.DataFrame(description_tfidf_array)
    description_tfidf_df.columns = list(map(lambda x : "description_" + str(x), description_tfidf_df.columns))
    df = pd.merge(df, description_tfidf_df , left_index=True, right_index=True)
    
    description_story_processed_tfidf_array = tfidf_fit_description_story_processed.transform(df['description_story_processed']).toarray()
    description_story_tfidf_df = pd.DataFrame(description_story_processed_tfidf_array)
    description_story_tfidf_df.columns = list(map(lambda x : "description_story_" + str(x), description_story_tfidf_df.columns))
    df = pd.merge(df, description_story_tfidf_df , left_index=True, right_index=True)
    
    description_risks_processed_tfidf_array = tfidf_fit_description_risks_processed.transform(df['description_risks_processed']).toarray()
    description_risks_tfidf_df = pd.DataFrame(description_risks_processed_tfidf_array)
    description_risks_tfidf_df.columns = list(map(lambda x : "description_risks_" + str(x), description_risks_tfidf_df.columns))
    df = pd.merge(df, description_risks_tfidf_df , left_index=True, right_index=True)
    
    return df


## Rewards Features

In [8]:
# Remove rewards that got cut off while scraping and closes the dictionary
def create_rewards_tiers(df):
    # df["rewards"] = df["rewards"].apply(lambda x : clean_rewards(x))
    df["reward_tiers"] = df["rewards"].apply(lambda x : len(ast.literal_eval(x)))
    df = move_reward_tiers(df)
    return df

def create_min_max_reward(df):
    df["min_reward"] = 0
    df["max_reward"] = 0

    for i in range(len(df)):
        # if i % 100 == 0:
        #     print(i)
        all_reward_amount = []
        reward = df.iloc[i, 5]
        dict_list = ast.literal_eval(reward)
        for dictionary in dict_list:
            values_string = str(dictionary.values())
            reward_title_SGD = re.search(r"(S)?\$\s?\d{1,3}(,\d{1,3})?", values_string) #S$
            if reward_title_SGD:
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title_SGD.group())
            else:
                 print("Error: No reward in SGD found!")   

            if reward_amount is not None:
                integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money
                all_reward_amount.append(int(integer_amount))
            else:
                pledged_without_reward = re.search(r"Pledge without a reward", values_string) # check if have "Pledge without a reward"
                if pledged_without_reward:
                    all_reward_amount.append(0)
        if len(all_reward_amount) > 0:
            df.loc[i, "min_reward"] = min(all_reward_amount)
            df.loc[i, "max_reward"] = max(all_reward_amount)
    print("create min max reward done, now moving columns")
    df = move_min_max_reward(df)
    return df

# Rearange reward_tiers column to the right of rewards
def move_reward_tiers(df):
    cols = df.columns.tolist()
    cols = cols[:6] + [cols[-1]] + cols[6:-1]
    df = df[cols]
    return df

# Rearange min and max reward columns to the right of reward_tiers
def move_min_max_reward(df):
    cols = df.columns.tolist()
    cols = cols[:7] + cols[-2:] + cols[7:-2]
    df = df[cols]
    return df

# Remove backer count in rewards to prevent data leakage
def remove_backers(df):
    new_df = df.copy()
    for i in range(len(df)):
        reward = df.iloc[i, 5]
        dict_list = ast.literal_eval(reward)
        new_list = []

        for dictionary in dict_list:
            values_string = str(dictionary.values())
            match_result = re.search(r"\d+ backer(s)?", values_string) #\nxxx backer
            start_index = match_result.span()[0]
            new_reward = values_string[14:start_index] #Remove "dict_values" and backer-count related words
            new_list.append({'rewards': new_reward})

        new_df.iloc[i,5] = str(new_list)
    return new_df

<>:27: DeprecationWarning: invalid escape sequence '\d'
<>:27: DeprecationWarning: invalid escape sequence '\d'
C:\Users\bandy\AppData\Local\Temp\ipykernel_13136\2418721434.py:27: DeprecationWarning: invalid escape sequence '\d'
  integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money


## Sentiment Features

In [9]:
'''
Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement. Subjective sentences generally refer to personal opinion, emotion or judgment whereas objective refers to factual information. Subjectivity is also a float which lies in the range of [0,1].
'''
def generate_sentiment_features(df):
    df = df.dropna(subset=['description_story_processed', 'description_risks_processed', 'description_processed', 'rewards_processed']) # NOTE: put at top with other dropnas from other features?
    df["description_story_polarity"] = df["description_story_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_story_subjectivity"] = df["description_story_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["description_polarity"] = df["description_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_subjectivity"] = df["description_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["description_risks_polarity"] = df["description_risks_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_risks_subjectivity"] = df["description_risks_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["rewards_polarity"] = df["rewards_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["rewards_subjectivity"] = df["rewards_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)          
    return df

In [10]:
def generate_word_count_features(df): # omitted description due to word limit, word count likely similar for all projects
    df['description_story_word_count'] = df["description_story_processed"].apply(lambda x: len(str(x).split(" ")))
    df['description_risks_word_count'] = df["description_risks_processed"].apply(lambda x: len(str(x).split(" ")))
    df['rewards_word_count'] = df["rewards_processed"].apply(lambda x: len(str(x).split(" ")))
    return df

## One-hot Encoding of Categorical Variables

when to drop one of the OHE columns: 
https://stats.stackexchange.com/questions/231285/dropping-one-of-the-columns-when-using-one-hot-encoding

In [11]:
def ohe_transform(enc, col, df):
    transformed = enc.transform(df[col].to_numpy().reshape(-1, 1))
    #Create a Pandas DataFrame of the hot encoded column
    ohe_df = pd.DataFrame(transformed, columns=enc.get_feature_names_out())
    #concat with original data
    data = pd.concat([df, ohe_df], axis=1)#.drop([col], axis=1)
    return data

def ohe_fit(col, df):
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    enc.fit(df[col].unique().reshape(-1, 1))
    return (ohe_transform(enc, col, df), enc)

hold out on topic modelling first because it is unsupervised algorithm

<h2> Generate Duration Feature </h2>

In [12]:
def generate_duration_features(df):
    df['deadline'] = df['deadline'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['launched_at'] = df['launched_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['created_at'] = df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['funding_duration_in_days'] = (df['deadline'] - df['launched_at'].dt.tz_localize(None))
    df['funding_duration_in_days'] = df['funding_duration_in_days'].apply(lambda x: x / timedelta(days=1))
    df['preparation_duration_in_days'] = (df['launched_at'] - df['created_at'].dt.tz_localize(None))
    df['preparation_duration_in_days'] = df['preparation_duration_in_days'].apply(lambda x: x / timedelta(days=1))
    
    return df

## Text Competitor Feature

In [13]:
'''
text competitors: inverse originality of text corpus. 

If score is higher means how the project owner 
words the project is very similar to all the other projects out there, meaning more competitive
because the wordings of the project is very 'unoriginal'
'''

def generate_text_competitor(df):
    # combine all text into 1 corpus 
    df['corpus'] = df['description_processed'] + df['description_story_processed'] + df['description_risks_processed']
    # generate a sparse matrix of tfidf 
    vect_corpus = TfidfVectorizer( 
            tokenizer=LemmatizeTokenizer(),
            lowercase=True,
            analyzer='word', 
            stop_words=stopwords.words('english') # remove stopwords
            )
    tfidf_sparse = vect_corpus.fit_transform(df['corpus'])
    # calculate cosine similarity based on the tfidf 
    cos_sim_dense = cosine_similarity(tfidf_sparse)
    # remove diagonal - A project is exactly the same as itself 
    cos_sim_dense_i_removed = cos_sim_dense[~np.eye(cos_sim_dense.shape[0],dtype=bool)].reshape(cos_sim_dense.shape[0],-1)
    # taking median as the similarity threshold - if greater than equal to median, means similar to that specific project 
    threshold = np.median(cos_sim_dense_i_removed.flatten())
    # if cosine similarity is above threshold return 1, then sum to get similar counts 
    df['text_similarity_score'] = np.apply_along_axis(np.sum, 1, cos_sim_dense_i_removed >= threshold)

    # sns.histplot(result['text_similarity_score'])

    return (df, vect_corpus, threshold)

def generate_text_competitor_test(vectorizer, df, threshold):
    # combine all text into 1 corpus 
    df['corpus'] = df['description_processed'] + df['description_story_processed'] + df['description_risks_processed']
    tfidf_sparse = vectorizer.transform(df['corpus'])
    # calculate cosine similarity based on the tfidf 
    cos_sim_dense = cosine_similarity(tfidf_sparse)
    # remove diagonal - A project is exactly the same as itself 
    cos_sim_dense_i_removed = cos_sim_dense[~np.eye(cos_sim_dense.shape[0],dtype=bool)].reshape(cos_sim_dense.shape[0],-1)
    # taking median as the similarity threshold - if greater than equal to median, means similar to that specific project 
    # threshold from training
    # if cosine similarity is above threshold return 1, then sum to get similar counts 
    df['text_similarity_score'] = np.apply_along_axis(np.sum, 1, cos_sim_dense_i_removed >= threshold)

    # sns.histplot(result['text_similarity_score'])

    return df

## Category and goal competitor feature

In [14]:
'''
Category and goal competitor: projects in the same category, that launched in the same month and in the same goal bin
'''
def generate_cat_goal_competitor(df):
    #df['launched_at'] = df['launched_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['launched_month']= df['launched_at'].dt.month
    df['launched_year']= df['launched_at'].dt.year
    df['cat_goal_bin'] = df.groupby(['category'])['goal'].transform(
                        lambda x: pd.qcut(x, [0, .33, .67, 1]))
    
    competitors = df.groupby(['category','launched_year','launched_month','cat_goal_bin']).count()
    competitors['competitiors_count'] = competitors['name']
    competitors = competitors[['competitiors_count']]
    competitors.reset_index(inplace=True)
    df = pd.merge(df, competitors, on = ['category', 'launched_year', 'launched_month','cat_goal_bin'], how = 'left')

    return df

## Similar success rate feature

In [15]:
'''
Success rate for similar projects belonging to a particular category, with similar goal and launched in the same year.
'''
def generate_similiar_success_rate(df):
    df_success = df[df['state'] == 1]
    df_success = df_success.groupby(['category','launched_year','cat_goal_bin']).count()
    df_success['success_count'] = df_success['name']
    df_success = df_success[['success_count']]
    df_success.reset_index(inplace=True)
    df = pd.merge(df, df_success, on = ['category', 'launched_year', 'cat_goal_bin'], how = 'left')

    df_total = df.groupby(['category','launched_year','cat_goal_bin']).count()
    df_total['total_count'] = df_total['name']
    df_total = df_total[['total_count']]
    df_total.reset_index(inplace=True)
    df = pd.merge(df, df_total, on = ['category', 'launched_year', 'cat_goal_bin'], how = 'left')
    df['success_rate'] = df['success_count'] / df['total_count']
    df['success_rate'] = df['success_rate'].fillna(0)
    return df

## Combine all feature generating functions

In [15]:
result = create_rewards_tiers(df)
print("Rewards tiers done")
result = create_min_max_reward(result)
print("min max rewards done")
result = remove_backers(result)
print("remove backers done")

result = clean_text(result)
print("clean text done")
result['has_video'] = result['video'].apply(lambda x: 0 if pd.isnull(x) else 1)

result, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed, tfidf_feature_names = generate_nlp_features(result)
print("generate_nlp_features done")

result = generate_sentiment_features(result)
print("generate_sentiment_features done")
result = generate_word_count_features(result)
print("generate_word_count_features done")
result = generate_duration_features(result)
print("generate_duration_features done")
result, vect_corpus, threshold = generate_text_competitor(result)
print("generate_text_competitor done")
result = generate_cat_goal_competitor(result)
print("generate_cat_goal_competitor done")
result = generate_similiar_success_rate(result)
print("generate_similiar_success_rate done")

result, category_encoder = ohe_fit('category', result) #use encoder to fit train data
print("ohe_fit category done")
result, location_encoder = ohe_fit('location', result) #use encoder to fit train data
print("ohe_fit location done")

Rewards tiers done


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_39674/2418721434.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["min_reward"] = 0
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_39674/2418721434.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["max_reward"] = 0


create min max reward done, now moving columns
min max rewards done
clean text done


/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn(
/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/feature_ext

generate_nlp_features done
generate_sentiment_features done
generate_word_count_features done
generate_duration_features done


/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn(


generate_text_competitor done
generate_cat_goal_competitor done
generate_similiar_success_rate done
ohe_fit category done
ohe_fit location done


In [16]:
# tfidf_feature_names

In [17]:
# with open("tfidf_feature_names.txt", 'w') as f:
#     json.dump(tfidf_feature_names,f)

## Drop Unnecessary Features, Feature Scaling

In [18]:
to_drop = ['rewards', 'deadline', 'launched_at', 'rewards_processed', 'created_at',
           'description_processed', 'description_story_processed','description_risks_processed',
           'id', 'name', 'description', 'description_story', 'description_risks', 'video',
          'location', 'category', 'corpus', 'launched_month', 'launched_year', 'cat_goal_bin', 'success_count', 'total_count']

final_result = result.drop(to_drop, axis = 1)

#need feature scaling
scaler = MinMaxScaler() 
need_scaling = ['reward_tiers', 'min_reward', 'max_reward', 'goal', 'text_similarity_score',
               'description_story_polarity', 'description_story_subjectivity', 'description_polarity',
               'description_subjectivity', 'description_risks_polarity', 'description_risks_subjectivity',
               'rewards_polarity', 'rewards_subjectivity', 'description_story_word_count',
               'description_risks_word_count', 'rewards_word_count', 'funding_duration_in_days',
               'preparation_duration_in_days', 'competitiors_count', 'success_rate']
final_result[need_scaling] = scaler.fit_transform(final_result[need_scaling])

## Apply same feature engineering on Test Data

In [19]:
#test = pd.read_csv("kickstarter_test.csv")
test = pd.read_csv("/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/test/kickstarter_test.csv")
test = create_rewards_tiers(test)
test = create_min_max_reward(test)

test = clean_text(test)
test['has_video'] = test['video'].apply(lambda x: 0 if pd.isnull(x) else 1)

test = nlp_transform_test(test, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed)

test = generate_sentiment_features(test)
test = generate_word_count_features(test)
test = generate_duration_features(test)
test = generate_text_competitor_test(vect_corpus, test, threshold)
test = generate_cat_goal_competitor(test)
test = generate_similiar_success_rate(test)

#encoders
test = ohe_transform(category_encoder, 'category', test)
test = ohe_transform(location_encoder, 'location', test)

#drop and scale 
final_test = test.drop(to_drop, axis=1)
final_test[need_scaling] = scaler.transform(final_test[need_scaling])

create min max reward done, now moving columns


In [34]:
import pickle
tfidf_feature_names

with open('tfidf_feature_names.pkl', 'wb') as f:
    pickle.dump(tfidf_feature_names, f)

In [21]:
print(len(final_test.columns))
print(len(final_result.columns))

for i in result.columns:
    if i not in test.columns:
        print(i)

579
579


In [22]:
final_result.columns

Index(['reward_tiers', 'min_reward', 'max_reward', 'goal', 'state',
       'staff_pick', 'has_video', 'rewards_0', 'rewards_1', 'rewards_2',
       ...
       'x0_UY', 'x0_VE', 'x0_VN', 'x0_VU', 'x0_WS', 'x0_XK', 'x0_YE', 'x0_ZA',
       'x0_ZW', 'x0_nan'],
      dtype='object', length=579)

## Export

In [23]:
import time 

final_result.to_csv(f'/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/train/kickstarter_train_final_{time.strftime("%Y%m%d-%H%M%S")}.csv', index=False)
final_test.to_csv(f'/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/test/kickstarter_test_final_{time.strftime("%Y%m%d-%H%M%S")}.csv', index=False)


In [24]:
#since there could be more than one categories for each project, create new features to split the categories.
# df['new_category'] = df.category.str.split("/", expand=False)
# split_cat = pd.DataFrame(df['new_category'].tolist(), columns=['category1', 'category2'])
# #each project should at least have 1 category, 'category2' can be "None". 
# #'category1' being the main category for the project.
# df = pd.concat([df, split_cat], axis=1)
# df.head()t

In [25]:

pd.set_option('display.max_columns', None)

In [26]:
final_test.reward_tiers.value_counts()

0.033333    409
0.050000    388
0.058333    367
0.041667    364
0.025000    337
0.066667    291
0.075000    286
0.016667    274
0.000000    259
0.083333    199
0.091667    177
0.008333    160
0.100000    160
0.108333    117
0.116667    106
0.125000     90
0.133333     64
0.141667     48
0.150000     40
0.166667     29
0.158333     28
0.175000     27
0.191667     24
0.183333     17
0.216667     14
0.233333     11
0.200000     11
0.208333      9
0.241667      8
0.225000      8
0.250000      6
0.291667      5
0.275000      5
0.258333      4
0.266667      3
0.283333      3
0.350000      2
0.341667      2
0.325000      2
0.300000      2
0.358333      2
0.333333      2
0.375000      1
0.483333      1
0.833333      1
0.433333      1
0.308333      1
0.558333      1
0.491667      1
0.550000      1
0.316667      1
Name: reward_tiers, dtype: int64

In [27]:
final_test.head()

,reward_tiers,min_reward,max_reward,goal,state,staff_pick,has_video,rewards_0,rewards_1,rewards_2,rewards_3,rewards_4,rewards_5,rewards_6,rewards_7,rewards_8,rewards_9,rewards_10,rewards_11,rewards_12,rewards_13,rewards_14,rewards_15,rewards_16,rewards_17,rewards_18,rewards_19,rewards_20,rewards_21,rewards_22,rewards_23,rewards_24,rewards_25,rewards_26,rewards_27,rewards_28,rewards_29,rewards_30,rewards_31,rewards_32,rewards_33,rewards_34,rewards_35,rewards_36,rewards_37,rewards_38,rewards_39,rewards_40,rewards_41,rewards_42,rewards_43,rewards_44,rewards_45,rewards_46,rewards_47,rewards_48,rewards_49,rewards_50,rewards_51,rewards_52,rewards_53,rewards_54,rewards_55,rewards_56,rewards_57,rewards_58,rewards_59,rewards_60,rewards_61,rewards_62,rewards_63,rewards_64,rewards_65,rewards_66,rewards_67,rewards_68,rewards_69,rewards_70,rewards_71,rewards_72,rewards_73,rewards_74,rewards_75,rewards_76,rewards_77,rewards_78,rewards_79,rewards_80,rewards_81,rewards_82,rewards_83,rewards_84,rewards_85,rewards_86,rewards_87,rewards_88,rewards_89,rewards_90,rewards_91,rewards_92,rewards_93,rewards_94,rewards_95,rewards_96,rewards_97,rewards_98,rewards_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,description_story_0,description_story_1,description_story_2,description_story_3,description_story_4,description_story_5,description_story_6,description_story_7,description_story_8,description_story_9,description_story_10,description_story_11,description_story_12,description_story_13,description_story_14,description_story_15,description_story_16,description_story_17,description_story_18,description_story_19,description_story_20,description_story_21,description_story_22,description_story_23,description_story_24,description_story_25,description_story_26,description_story_27,description_story_28,description_story_29,description_story_30,description_story_31,description_story_32,description_story_33,description_story_34,description_story_35,description_story_36,description_story_37,description_story_38,description_story_39,description_story_40,description_story_41,description_story_42,description_story_43,description_story_44,description_story_45,description_story_46,description_story_47,description_story_48,description_story_49,description_story_50,description_story_51,description_story_52,description_story_53,description_story_54,description_story_55,description_story_56,description_story_57,description_story_58,description_story_59,description_story_60,description_story_61,description_story_62,description_story_63,description_story_

In [28]:
final_result.head()

,reward_tiers,min_reward,max_reward,goal,state,staff_pick,has_video,rewards_0,rewards_1,rewards_2,rewards_3,rewards_4,rewards_5,rewards_6,rewards_7,rewards_8,rewards_9,rewards_10,rewards_11,rewards_12,rewards_13,rewards_14,rewards_15,rewards_16,rewards_17,rewards_18,rewards_19,rewards_20,rewards_21,rewards_22,rewards_23,rewards_24,rewards_25,rewards_26,rewards_27,rewards_28,rewards_29,rewards_30,rewards_31,rewards_32,rewards_33,rewards_34,rewards_35,rewards_36,rewards_37,rewards_38,rewards_39,rewards_40,rewards_41,rewards_42,rewards_43,rewards_44,rewards_45,rewards_46,rewards_47,rewards_48,rewards_49,rewards_50,rewards_51,rewards_52,rewards_53,rewards_54,rewards_55,rewards_56,rewards_57,rewards_58,rewards_59,rewards_60,rewards_61,rewards_62,rewards_63,rewards_64,rewards_65,rewards_66,rewards_67,rewards_68,rewards_69,rewards_70,rewards_71,rewards_72,rewards_73,rewards_74,rewards_75,rewards_76,rewards_77,rewards_78,rewards_79,rewards_80,rewards_81,rewards_82,rewards_83,rewards_84,rewards_85,rewards_86,rewards_87,rewards_88,rewards_89,rewards_90,rewards_91,rewards_92,rewards_93,rewards_94,rewards_95,rewards_96,rewards_97,rewards_98,rewards_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,description_story_0,description_story_1,description_story_2,description_story_3,description_story_4,description_story_5,description_story_6,description_story_7,description_story_8,description_story_9,description_story_10,description_story_11,description_story_12,description_story_13,description_story_14,description_story_15,description_story_16,description_story_17,description_story_18,description_story_19,description_story_20,description_story_21,description_story_22,description_story_23,description_story_24,description_story_25,description_story_26,description_story_27,description_story_28,description_story_29,description_story_30,description_story_31,description_story_32,description_story_33,description_story_34,description_story_35,description_story_36,description_story_37,description_story_38,description_story_39,description_story_40,description_story_41,description_story_42,description_story_43,description_story_44,description_story_45,description_story_46,description_story_47,description_story_48,description_story_49,description_story_50,description_story_51,description_story_52,description_story_53,description_story_54,description_story_55,description_story_56,description_story_57,description_story_58,description_story_59,description_story_60,description_story_61,description_story_62,description_story_63,description_story_

In [29]:
result.head()

,id,name,description,description_story,description_risks,rewards,reward_tiers,min_reward,max_reward,category,goal,deadline,location,state,staff_pick,video,launched_at,created_at,rewards_processed,description_processed,description_story_processed,description_risks_processed,has_video,rewards_0,rewards_1,rewards_2,rewards_3,rewards_4,rewards_5,rewards_6,rewards_7,rewards_8,rewards_9,rewards_10,rewards_11,rewards_12,rewards_13,rewards_14,rewards_15,rewards_16,rewards_17,rewards_18,rewards_19,rewards_20,rewards_21,rewards_22,rewards_23,rewards_24,rewards_25,rewards_26,rewards_27,rewards_28,rewards_29,rewards_30,rewards_31,rewards_32,rewards_33,rewards_34,rewards_35,rewards_36,rewards_37,rewards_38,rewards_39,rewards_40,rewards_41,rewards_42,rewards_43,rewards_44,rewards_45,rewards_46,rewards_47,rewards_48,rewards_49,rewards_50,rewards_51,rewards_52,rewards_53,rewards_54,rewards_55,rewards_56,rewards_57,rewards_58,rewards_59,rewards_60,rewards_61,rewards_62,rewards_63,rewards_64,rewards_65,rewards_66,rewards_67,rewards_68,rewards_69,rewards_70,rewards_71,rewards_72,rewards_73,rewards_74,rewards_75,rewards_76,rewards_77,rewards_78,rewards_79,rewards_80,rewards_81,rewards_82,rewards_83,rewards_84,rewards_85,rewards_86,rewards_87,rewards_88,rewards_89,rewards_90,rewards_91,rewards_92,rewards_93,rewards_94,rewards_95,rewards_96,rewards_97,rewards_98,rewards_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,description_story_0,description_story_1,description_story_2,description_story_3,description_story_4,description_story_5,description_story_6,description_story_7,description_story_8,description_story_9,description_story_10,description_story_11,description_story_12,description_story_13,description_story_14,description_story_15,description_story_16,description_story_17,description_story_18,description_story_19,description_story_20,description_story_21,description_story_22,description_story_23,description_story_24,description_story_25,description_story_26,description_story_27,description_story_28,description_story_29,description_story_30,description_story_31,description_story_32,description_story_33,description_story_34,description_story_35,description_story_36,description_story_37,description_story_38,description_story_39,description_story_40,description_story_41,description_story_42,description_story_43,description_story_44,description_story_45,description_story_46,description_story_47,description_story_48,description_story_49,description_story_50,description_story_51,description_story_52,description_story_53,description_

In [30]:
#fuck = pd.read_csv('../kickstarter_train_final_20221021-185107.csv')

In [31]:
fuck.head()

NameError: name 'fuck' is not defined

In [ ]:
t = pd.read_csv("/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/test/kickstarter_test.csv")
t['rewards']